**CSE 512 Spring 2021 - Machine Learning - Homework 4**


---


Your Name: Irfan Ahmed \\
Solar ID : 113166464 \\
NetID Email: irfan.ahmed@stonybrook.edu

In [ ]:
# Mount your google drive where you've saved your assignment folder
from google.colab import drive
drive.mount('/content/gdrive') 

Mounted at /content/gdrive


In [ ]:
%cd /content/gdrive/MyDrive/Colab\ Notebooks/CSE512_ML/HW4/

/content/gdrive/MyDrive/Colab Notebooks/CSE512_ML/HW4


In [ ]:
!pip install xgboost

In [ ]:
from xgboost import XGBClassifier
import numpy as np
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import average_precision_score
from sklearn import metrics
from sklearn.model_selection import RandomizedSearchCV
from sklearn.impute import SimpleImputer
import collections
import pandas as pd
import statistics

In [ ]:
#!unzip HW4_q2.zip

In [ ]:
%cd HW4_q2/

/content/gdrive/MyDrive/Colab Notebooks/CSE512_ML/HW4/HW4_q2


age: continuous 


workclass: Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked.


fnlwgt: continuous.


education: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool.


education-num: continuous.


marital-status: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse.


occupation: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces.


relationship: Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried.


race: White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black.


sex: Female, Male.


capital-gain: continuous.


capital-loss: continuous.


hours-per-week: continuous.


native-country: United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands.


In [ ]:
col_names = {'age': 0, 'workclass': 1, 'fnlwgt': 2, 'education': 3, 'education-num': 4 , 'marital-status' : 5, 'occupation' : 6, 'relationship' : 7,
             'race': 8, 'sex': 9, 'capital-gain': 10, 'capital-loss': 11, 'hours-per-week':12, 'native-country':13 }
continuous_cols = [0,2,4,10,11,12]
categorical_cols = [1,3,5,6,7,8,9,13,14]             

In [ ]:
train_data = pd.read_csv('adult.data', header=None).to_numpy()
test_data = pd.read_csv('adult.test', header=None).to_numpy()

In [ ]:
for i in continuous_cols:
  train_data[:,i].astype(np.float)
  test_data[:,i].astype(np.float)

In [ ]:
for i in categorical_cols:
  OE = OrdinalEncoder()
  
  Imputer = SimpleImputer(missing_values=' ?', strategy='most_frequent')
  train_data[:,i] = Imputer.fit_transform(train_data[:,i].reshape(-1,1)).reshape(1,-1)
  test_data[:,i] = Imputer.transform(test_data[:,i].reshape(-1,1)).reshape(1,-1)

  train_data[:,i] = OE.fit_transform(train_data[:,i].reshape(-1,1)).reshape(1,-1)
  test_data[:,i] = OE.transform(test_data[:,i].reshape(-1,1)).reshape(1,-1)

In [ ]:
X_train, y_train = train_data[:, :-1] , train_data[:, -1].astype(np.float)
X_test, y_test = test_data[:, :-1] , test_data[:, -1].astype(np.float)
print(y_test.shape)

(16281,)


In [ ]:
model = XGBClassifier(max_depth=5)
model.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=5,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [ ]:
y_pred = model.predict(X_test)
print(y_pred, y_test)

[0. 0. 0. ... 1. 0. 1.] [0. 0. 1. ... 0. 0. 1.]


In [ ]:
def print_scores(y1,y2):
  y1,y2 = y1.astype(np.float), y2.astype(np.float)
  accuracy = accuracy_score(y1,y2)
  cf_matrix = confusion_matrix(y1,y2)
  avg_precision = average_precision_score(y1,y2)

  print('\n Accuracy : ' + str(accuracy))
  print('\n Confusion matrix :\n ' + str(cf_matrix))
  print('\n Average Precision Score : ' + str(avg_precision))

In [ ]:
accuracy = accuracy_score(y_test.astype(np.float), y_pred.astype(np.float))
print_scores(y_test, y_pred)


 Accuracy : 0.8734107241569928

 Confusion matrix :
 [[11798   637]
 [ 1424  2422]]

 Average Precision Score : 0.5860722346117625


In [ ]:
max_depth_val = [3,4,5,6,7,8,9,10]
kf = KFold(n_splits=10)


for i in range(len(max_depth_val)):
  my_list = []
  for train_index, test_index in kf.split(X_train):
    X_train_Kfold, X_test_Kfold = X_train[train_index], X_train[test_index]
    y_train_Kfold, y_test_Kfold = y_train[train_index], y_train[test_index]
    model = XGBClassifier(max_depth = max_depth_val[i])
    model.fit(X_train_Kfold, y_train_Kfold)
    y_pred_Kfold = model.predict(X_test_Kfold)
    my_list.append(accuracy_score(y_test_Kfold, y_pred_Kfold))
  
  print("\nFor max_depth = " + str(max_depth_val[i]) + " - mean : " + str(statistics.mean(my_list)) + " , variance : " + str(statistics.variance(my_list)))



For max_depth = 3 - mean : 0.8624123226556447 , variance : 2.9060335094163053e-05

For max_depth = 4 - mean : 0.8684624366041315 , variance : 2.938011737040485e-05

For max_depth = 5 - mean : 0.8716257706893261 , variance : 3.796171406888943e-05

For max_depth = 6 - mean : 0.8724549712997671 , variance : 3.035801285392901e-05

For max_depth = 7 - mean : 0.8725777554147219 , variance : 3.8015468434759706e-05

For max_depth = 8 - mean : 0.8723935179492441 , variance : 3.5444160081458636e-05

For max_depth = 9 - mean : 0.8730385282427039 , variance : 2.7687810529702096e-05

For max_depth = 10 - mean : 0.8715336236674892 , variance : 2.637496412355618e-05


In [ ]:
param_test = {
  'max_depth':[4,5,6,7,8,9],
  'reg_alpha':[0, 0.001, 0.005, 0.01, 0.05],
  'learning_rate':[0.01,0.05,0.1,0.15,0.2,0.25,0.3],
  'min_child_weight':[4,5,6],
  'gamma':[i/10.0 for i in range(0,5)],
  'subsample':[i/10.0 for i in range(6,10)],
  'colsample_bytree':[i/10.0 for i in range(6,10)]
}

In [ ]:
model = XGBClassifier()
clf = RandomizedSearchCV(model, param_test, scoring='roc_auc',cv=10)
search = clf.fit(X_train, y_train.astype(np.float))

In [ ]:
search.best_params_

{'colsample_bytree': 0.6,
 'gamma': 0.4,
 'learning_rate': 0.15,
 'max_depth': 7,
 'min_child_weight': 4,
 'reg_alpha': 0.001,
 'subsample': 0.8}

In [ ]:
model = XGBClassifier(max_depth=7,learning_rate=0.15,gamma=0.4,colsample_bytree=0.6,min_child_weight=4,reg_alpha=0.001,subsample=0.8)
model.fit(X_train,y_train)
y_pred = model.predict(X_test)
print_scores(y_test,y_pred)


 Accuracy : 0.8726122474049506

 Confusion matrix :
 [[11698   737]
 [ 1337  2509]]

 Average Precision Score : 0.5863674999321782
